In [1]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=500, window=50, min_count=1, workers=8)

In [2]:
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("my_doc2vec_model")
model.save(fname)
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

In [3]:
import pandas as pd

data = pd.read_csv('processed_data_prime.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6076 entries, 0 to 6075
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   article_number        6076 non-null   int64 
 1   processed_text        6076 non-null   object
 2   constitution_article  2019 non-null   object
 3   crpc_article          429 non-null    object
 4   ipc_article           518 non-null    object
 5   label                 6076 non-null   int64 
 6   split                 6076 non-null   object
dtypes: int64(2), object(5)
memory usage: 332.4+ KB


In [4]:
data = data.reset_index()
data

,index,article_number,processed_text,constitution_article,crpc_article,ipc_article,label,split
0,0,0,F NARIMAN J Leave granted In 2008 the Punjab ...,~~Equality of opportunity in matters of public...,NaN,NaN,1,train
1,1,1,S THAKUR J Leave granted These appeals are di...,~~Recruitment and conditions of service of per...,NaN,NaN,0,train
2,2,2,Markandey Katju J Leave granted Heard learned...,NaN,~~Order for maintenance of wives children and ...,NaN,1,train
3,3,3,ALTAMAS KABIRJ Leave granted The question whe...,~~Power of High Courts to issue certain writs ...,~~Compounding of offences The offences punisha...,~~Whoever commits criminal breach of trust sha...,1,train
4,4,4,CIVIL APPEAL NO 598 OF 2007 K MATHUR J This a...,~~Jurisdiction of existing High Courts Subject...,NaN,NaN,1,train
...,...,...,...,...,...,...,...,...
6071,6071,6071,civil appellate jurisdiction civil appeal numb...,NaN,NaN,NaN,1,dev
6072,6072,6072,criminal appellate jurisdiction special leave\...,NaN,NaN,NaN,0,dev
6073,6073,6073,civil appellate jurisdiction civil appeal numb...,NaN,NaN,NaN,0,dev
6074,6074,6074,civil appellate jurisdiction civil appeal numb...,NaN,NaN,NaN,1,dev


In [5]:
from nltk import word_tokenize
def vectorize_processed_text(row):
    row['processed_text_vector'] =  model.infer_vector(word_tokenize(row['processed_text']))
    return row

pre_final = data[['article_number', 'processed_text', 'label', 'split']].apply(vectorize_processed_text, axis=1)
pre_final

,article_number,processed_text,label,split,processed_text_vector
0,0,F NARIMAN J Leave granted In 2008 the Punjab ...,1,train,"[-0.0009832437, -0.0006033854, -0.00044061168,..."
1,1,S THAKUR J Leave granted These appeals are di...,0,train,"[-0.00038255434, -0.00082796277, -0.0008486368..."
2,2,Markandey Katju J Leave granted Heard learned...,1,train,"[-0.00016038705, -0.00024669245, 0.0006813637,..."
3,3,ALTAMAS KABIRJ Leave granted The question whe...,1,train,"[0.0007969142, 0.00046393072, 2.7258873e-05, -..."
4,4,CIVIL APPEAL NO 598 OF 2007 K MATHUR J This a...,1,train,"[0.00019346895, -0.00025453963, -0.00034809072..."
...,...,...,...,...,...
6071,6071,civil appellate jurisdiction civil appeal numb...,1,dev,"[0.0008156542, 0.0009509789, 7.1883915e-05, -0..."
6072,6072,criminal appellate jurisdiction special leave\...,0,dev,"[0.0008692498, 0.0006644974, -3.621268e-05, -0..."
6073,6073,civil appellate jurisdiction civil appeal numb...,0,dev,"[-0.00030640335, -0.00033959406, -0.0001059346..."
6074,6074,civil appellate jurisdiction civil appeal numb...,1,dev,"[0.00024632944, 0.0006550132, 1.3497829e-05, 0..."


In [6]:
import numpy as np
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [7]:
data[['processed_text', 'constitution_article', 'article_number']].dropna().groupby(by=['article_number'])

In [8]:
from nltk import word_tokenize
def divide_by_article_const(group):
    initial_data = group.iloc[0].to_dict()
    constitution = group['constitution_article'].iloc[0].split('~~')[1:]
    vector_text = model.infer_vector(word_tokenize(initial_data['processed_text']))

    counter = 0
    for const in constitution:
        counter = counter + 1
        new_data = initial_data.copy()

        vector_article = model.infer_vector(word_tokenize(const))
        new_data['processed_text_vector'] = vector_text
        new_data['constitution_article'] = const
        new_data['constitution_vector'] = vector_article
        new_data['cosine_constitution'] = cosine(vector_text, vector_article)

        group = group.append(new_data, ignore_index=True)

    return group

const = data[['processed_text', 'constitution_article', 'article_number']].dropna().groupby(['article_number']).apply(divide_by_article_const)
const

/var/folders/m6/ydbmjyvx5nz5ppn09gh7zky40000gn/T/ipykernel_37717/4245436520.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group = group.append(new_data, ignore_index=True)
/var/folders/m6/ydbmjyvx5nz5ppn09gh7zky40000gn/T/ipykernel_37717/4245436520.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group = group.append(new_data, ignore_index=True)
/var/folders/m6/ydbmjyvx5nz5ppn09gh7zky40000gn/T/ipykernel_37717/4245436520.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group = group.append(new_data, ignore_index=True)
/var/folders/m6/ydbmjyvx5nz5ppn09gh7zky40000gn/T/ipykernel_37717/4245436520.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

processed_text  \
article_number                                                        
0              0   F NARIMAN J Leave granted In 2008 the Punjab ...   
               1   F NARIMAN J Leave granted In 2008 the Punjab ...   
               2   F NARIMAN J Leave granted In 2008 the Punjab ...   
1              0   S THAKUR J Leave granted These appeals are di...   
               1   S THAKUR J Leave granted These appeals are di...   
...                                                             ...   
5077           0   1999  1  SCR 104 The Judgments of the Court w...   
               1   1999  1  SCR 104 The Judgments of the Court w...   
               2   1999  1  SCR 104 The Judgments of the Court w...   
5079           0   1999   4   Suppl SCR 360 ORDER The following ...   
               1   1999   4   Suppl SCR 360 ORDER The following ...   

                                               constitution_article  \
article_number                                                        
0              0  ~~Equality of opportunity in matters of public...   
               1  Equality of opportunity in matters of public e...   
               2  Equality before law The State shall not deny t...   
1              0  ~~Recruitment and conditions of service of per...   
               1  Recruitment and conditions of service of perso...   
...                                                             ...   
5077           0  ~~Protection against arrest and detention in c...   
               1  Protection against arrest and detention in cer...   
               2  Power of High Courts to issue certain writs No...   
5079           0  ~~Remedies for enforcement of rights conferred...   
               1  Remedies for enforcement of rights conferred b...   

                  article_number  \
article_number                     
0              0               0   
               1               0   
               2               0   
1              0               1   
               1               1   
...                          ...   
5077           0            5077   
               1            5077   
               2            5077   
5079           0            5079   
               1            5079   

                                              processed_text_vector  \
article_number                                                        
0              0                                                NaN   
               1  [-0.0009831183, -0.00059345027, -0.0004421392,...   
               2  [-0.0009831183, -0.00059345027, -0.0004421392,...   
1              0                                                NaN   
               1  [-0.00039040387, -0.0008521161, -0.00084340497...   
...                                                             ...   
5077           0                                                NaN   
               1  [4.2291405e-05, 0.0008433101, -0.00038924543, ...   
               2  [4.2291405e-05, 0.0008433101, -0.00038924543, ...   
5079           0                                                NaN   
               1  [0.00047064197, 0.00049728825, -0.00093129545,...   

                                                constitution_vector  \
article_number                                                        
0              0                                                NaN   
               1  [0.000796804, 0.0008490001, -0.0002705087, 0.0...   
               2  [-0.0005026783, 5.749643e-05, 0.000522226, -0....   
1              0                                                NaN   
               1  [-0.0005202818, -0.0009163675, 0.00022355306, ...   
...                                                             ...   
5077           0                                                NaN   
               1  [0.00072815997, 0.00039746365, -0.00035662059,...   
               2  [-0.0008169097, 0.0009687003, -0.00067808304, ...   
5079           0               

In [9]:
from nltk import word_tokenize
def divide_by_article_crpc(group):
    initial_data = group.iloc[0].to_dict()
    crpcs = group['crpc_article'].iloc[0].split('~~')[1:]
    vector_text = model.infer_vector(word_tokenize(initial_data['processed_text']))

    counter = 0
    for crpc_ in crpcs:
        counter = counter + 1
        new_data = initial_data.copy()

        vector_article = model.infer_vector(word_tokenize(crpc_))
        new_data['processed_text_vector'] = vector_text
        new_data['crpc_article'] = crpc_
        new_data['crpc_article_vector'] = vector_article
        new_data['cosine_crpc'] = cosine(vector_text, vector_article)

        group = group.append(new_data, ignore_index=True)

    return group

crpc = data[['processed_text', 'crpc_article', 'article_number']].dropna().groupby(['article_number']).apply(divide_by_article_crpc)
crpc

/var/folders/m6/ydbmjyvx5nz5ppn09gh7zky40000gn/T/ipykernel_37717/3021830307.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group = group.append(new_data, ignore_index=True)
/var/folders/m6/ydbmjyvx5nz5ppn09gh7zky40000gn/T/ipykernel_37717/3021830307.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group = group.append(new_data, ignore_index=True)
/var/folders/m6/ydbmjyvx5nz5ppn09gh7zky40000gn/T/ipykernel_37717/3021830307.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group = group.append(new_data, ignore_index=True)
/var/folders/m6/ydbmjyvx5nz5ppn09gh7zky40000gn/T/ipykernel_37717/3021830307.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

processed_text  \
article_number                                                        
2              0   Markandey Katju J Leave granted Heard learned...   
               1   Markandey Katju J Leave granted Heard learned...   
3              0   ALTAMAS KABIRJ Leave granted The question whe...   
               1   ALTAMAS KABIRJ Leave granted The question whe...   
               2   ALTAMAS KABIRJ Leave granted The question whe...   
...                                                             ...   
5995           1  criminal appellate jurisdiction criminal appea...   
5996           0  criminal appellate jurisdiction criminal appea...   
               1  criminal appellate jurisdiction criminal appea...   
6009           0  criminal appellate jurisdiction criminal appea...   
               1  criminal appellate jurisdiction criminal appea...   

                                                       crpc_article  \
article_number                                                        
2              0  ~~Order for maintenance of wives children and ...   
               1  Order for maintenance of wives children and pa...   
3              0  ~~Compounding of offences The offences punisha...   
               1  Compounding of offences The offences punishabl...   
               2  Saving of inherent powers of High Court Nothin...   
...                                                             ...   
5995           1  Practising pleader not to sit as Magistrate in...   
5996           0                                                 ~~   
               1                                                      
6009           0  ~~Report of police officer on completion of in...   
               1  Report of police officer on completion of inve...   

                  article_number  \
article_number                     
2              0               2   
               1               2   
3              0               3   
               1               3   
               2               3   
...                          ...   
5995           1            5995   
5996           0            5996   
               1            5996   
6009           0            6009   
               1            6009   

                                              processed_text_vector  \
article_number                                                        
2              0                                                NaN   
               1  [-0.00015469281, -0.00023798709, 0.00068276445...   
3              0                                                NaN   
               1  [0.0007969142, 0.00046393072, 2.7258873e-05, -...   
               2  [0.0007969142, 0.00046393072, 2.7258873e-05, -...   
...                                                             ...   
5995           1  [0.0007029376, 0.00052689825, -0.0008801441, 0...   
5996           0                                                NaN   
               1  [0.0008198155, 0.000102403814, 0.00020873487, ...   
6009           0                                                NaN   
               1  [0.00048279675, -0.00067295996, 0.00024942236,...   

                                                crpc_article_vector  \
article_number                                                        
2              0                                                NaN   
               1  [0.00086395844, 0.0004320318, 7.5529984e-05, -...   
3              0                                                NaN   
               1  [-0.00042701562, 0.00042720026, 0.0006856457, ...   
               2  [-0.0006820521, -0.0008467566, 0.0003117634, -...   
...                                                             ...   
5995           1  [0.0006614764, 0.0004177581, -0.0008282142, -0...   
5996           0                                                NaN   
               1  [0.00077756343, -0.00018606549, -0.0006253072,...   
6009           0               

In [16]:
from nltk import word_tokenize
def divide_by_article_ipc(group):
    initial_data = group.iloc[0].to_dict()
    ipcs = group['ipc_article'].iloc[0].split('~~')[1:]
    vector_text = model.infer_vector(word_tokenize(initial_data['processed_text']))

    counter = 0
    for ipc_ in ipcs:
        counter = counter + 1
        new_data = initial_data.copy()

        vector_article = model.infer_vector(word_tokenize(ipc_))
        new_data['processed_text_vector'] = vector_text
        new_data['ipc_article'] = ipc_
        new_data['ipc_article_vector'] = vector_article
        new_data['cosine_ipc'] = cosine(vector_text, vector_article)

        group = group.append(new_data, ignore_index=True)

    return group

ipc = data[['processed_text', 'ipc_article', 'article_number']].dropna().groupby(['article_number']).apply(divide_by_article_ipc)
ipc

/var/folders/m6/ydbmjyvx5nz5ppn09gh7zky40000gn/T/ipykernel_37717/3753818994.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group = group.append(new_data, ignore_index=True)
/var/folders/m6/ydbmjyvx5nz5ppn09gh7zky40000gn/T/ipykernel_37717/3753818994.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group = group.append(new_data, ignore_index=True)
/var/folders/m6/ydbmjyvx5nz5ppn09gh7zky40000gn/T/ipykernel_37717/3753818994.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group = group.append(new_data, ignore_index=True)
/var/folders/m6/ydbmjyvx5nz5ppn09gh7zky40000gn/T/ipykernel_37717/3753818994.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

processed_text  \
article_number                                                        
3              0   ALTAMAS KABIRJ Leave granted The question whe...   
               1   ALTAMAS KABIRJ Leave granted The question whe...   
               2   ALTAMAS KABIRJ Leave granted The question whe...   
               3   ALTAMAS KABIRJ Leave granted The question whe...   
               4   ALTAMAS KABIRJ Leave granted The question whe...   
...                                                             ...   
6010           0  criminal appellate jurisdiction criminal appea...   
               1  criminal appellate jurisdiction criminal appea...   
               2  criminal appellate jurisdiction criminal appea...   
6059           0  criminal appellate jurisdiction criminal appea...   
               1  criminal appellate jurisdiction criminal appea...   

                                                        ipc_article  \
article_number                                                        
3              0  ~~Whoever commits criminal breach of trust sha...   
               1  Whoever commits criminal breach of trust shall...   
               2  Whoever commits murder shall be punished with ...   
               3  Whoever cheats and thereby dishonestly induces...   
               4  Whoever except in the case provided for by sec...   
...                                                             ...   
6010           0  ~~Whoever commits murder shall be punished wit...   
               1  Whoever commits murder shall be punished with ...   
               2  Whoever knowing or having reason to believe th...   
6059           0  ~~Whoever commits murder shall be punished wit...   
               1  Whoever commits murder shall be punished with ...   

                  article_number  \
article_number                     
3              0               3   
               1               3   
               2               3   
               3               3   
               4               3   
...                          ...   
6010           0            6010   
               1            6010   
               2            6010   
6059           0            6059   
               1            6059   

                                              processed_text_vector  \
article_number                                                        
3              0                                                NaN   
               1  [0.0007969142, 0.00046393072, 2.7258873e-05, -...   
               2  [0.0007969142, 0.00046393072, 2.7258873e-05, -...   
               3  [0.0007969142, 0.00046393072, 2.7258873e-05, -...   
               4  [0.0007969142, 0.00046393072, 2.7258873e-05, -...   
...                                                             ...   
6010           0                                                NaN   
               1  [-0.00048697312, 0.0004895051, 0.00068926363, ...   
               2  [-0.00048697312, 0.0004895051, 0.00068926363, ...   
6059           0                                                NaN   
               1  [0.00014473652, -0.0006507586, -0.00049776793,...   

                                                 ipc_article_vector  \
article_number                                                        
3              0                                                NaN   
               1  [-0.00023069387, 0.00071619643, 0.00034159506,...   
               2  [-0.0003499089, -0.00018865854, 7.2042945e-05,...   
               3  [-0.00019394624, -0.0007354331, -5.9539318e-05...   
               4  [-0.0009890613, 0.0004525602, -0.0004944286, -...   
...                                                             ...   
6010           0                                                NaN   
               1  [-0.0003499089, -0.00018865854, 7.2042945e-05,...   
               2  [-9.098756e-05, -0.00020535123, 0.0009841664, ...   
6059           0               

In [12]:
const = const.drop(['article_number'], axis=1).reset_index()
const = const[const['level_1'] != 0]
const

,article_number,level_1,processed_text,constitution_article,processed_text_vector,constitution_vector,cosine_constitution
1,0,1,F NARIMAN J Leave granted In 2008 the Punjab ...,Equality of opportunity in matters of public e...,"[-0.0009831183, -0.00059345027, -0.0004421392,...","[0.000796804, 0.0008490001, -0.0002705087, 0.0...",-0.000198
2,0,2,F NARIMAN J Leave granted In 2008 the Punjab ...,Equality before law The State shall not deny t...,"[-0.0009831183, -0.00059345027, -0.0004421392,...","[-0.0005026783, 5.749643e-05, 0.000522226, -0....",-0.022057
4,1,1,S THAKUR J Leave granted These appeals are di...,Recruitment and conditions of service of perso...,"[-0.00039040387, -0.0008521161, -0.00084340497...","[-0.0005202818, -0.0009163675, 0.00022355306, ...",0.104040
5,1,2,S THAKUR J Leave granted These appeals are di...,Enforcement of decrees and orders of Supreme C...,"[-0.00039040387, -0.0008521161, -0.00084340497...","[-0.00047101796, -0.00079761556, -2.8206468e-0...",0.018013
6,1,3,S THAKUR J Leave granted These appeals are di...,Power of Parliament to amend the Constitution ...,"[-0.00039040387, -0.0008521161, -0.00084340497...","[-0.0007636164, 0.0009087316, 0.00018409168, 0...",-0.007085
...,...,...,...,...,...,...,...
5735,5075,1,1999 3 Suppl SCR 597 The Judgment of the C...,Rights of citizenship of certain persons who h...,"[0.0005851183, 0.0006615196, -0.00056849385, -...","[-0.00089419045, -0.000107291875, 0.0003528841...",-0.083147
5736,5075,2,1999 3 Suppl SCR 597 The Judgment of the C...,Definition In this Constitution unless the con...,"[0.0005851183, 0.0006615196, -0.00056849385, -...","[3.7139802e-05, -0.00085376855, 0.00015696377,...",-0.029315
5738,5077,1,1999 1 SCR 104 The Judgments of the Court w...,Protection against arrest and detention in cer...,"[4.2291405e-05, 0.0008433101, -0.00038924543, ...","[0.00072815997, 0.00039746365, -0.00035662059,...",0.010880
5739,5077,2,1999 1 SCR 104 The Judgments of the Court w...,Power of High Courts to issue certain writs No...,"[4.2291405e-05, 0.0008433101, -0.00038924543, ...","[-0.0008169097, 0.0009687003, -0.00067808304, ...",0.049734


In [13]:
crpc = crpc.drop(['article_number'], axis=1).reset_index()
crpc = crpc[crpc['level_1'] != 0]
crpc

,article_number,level_1,processed_text,crpc_article,processed_text_vector,crpc_article_vector,cosine_crpc
1,2,1,Markandey Katju J Leave granted Heard learned...,Order for maintenance of wives children and pa...,"[-0.00015469281, -0.00023798709, 0.00068276445...","[0.00086395844, 0.0004320318, 7.5529984e-05, -...",-0.054215
3,3,1,ALTAMAS KABIRJ Leave granted The question whe...,Compounding of offences The offences punishabl...,"[0.0007969142, 0.00046393072, 2.7258873e-05, -...","[-0.00042701562, 0.00042720026, 0.0006856457, ...",-0.094272
4,3,2,ALTAMAS KABIRJ Leave granted The question whe...,Saving of inherent powers of High Court Nothin...,"[0.0007969142, 0.00046393072, 2.7258873e-05, -...","[-0.0006820521, -0.0008467566, 0.0003117634, -...",-0.013734
6,13,1,2002 Supp 2 SCR 31 The Judgment of the Court...,Recording of confessions and statements Any Me...,"[-0.0004546254, 8.958689e-05, -0.0008090672, 0...","[-0.0003140463, 0.0003288144, -0.0009870988, -...",-0.028310
8,24,1,ORIGINAL JURISDICTION Petition No 54 of 1953 P...,Summary procedure for trial for giving false e...,"[-2.1779537e-05, -0.00066729443, 0.0004547963,...","[-0.00092530705, 0.00020640029, 0.0004518886, ...",0.065163
...,...,...,...,...,...,...,...
1148,5928,2,criminal appellate jurisdiction criminal appea...,Offence triable where act is done or consequen...,"[0.00047262944, -0.0008629642, -0.00052978634,...","[0.00084999023, 0.00038852514, -3.3362805e-05,...",0.008524
1150,5956,1,criminal appellate jurisdiction criminal appea...,Power to take bond for appearance When any per...,"[-0.00041264, 0.0006905154, -0.00074751815, 0....","[0.0008500668, -0.00012428033, 0.0006465589, -...",0.014898
1152,5995,1,criminal appellate jurisdiction criminal appea...,Practising pleader not to sit as Magistrate in...,"[0.0007029376, 0.00052689825, -0.0008801441, 0...","[0.0006614764, 0.0004177581, -0.0008282142, -0...",0.096582
1154,5996,1,criminal appellate jurisdiction criminal appea...,,"[0.0008198155, 0.000102403814, 0.00020873487, ...","[0.00077756343, -0.00018606549, -0.0006253072,...",-0.002936


In [18]:
ipc = ipc.drop(['article_number'], axis=1).reset_index()
ipc = ipc[ipc['level_1'] != 0]  # first row contains rough data with ~~ splitter
ipc

,article_number,level_1,processed_text,ipc_article,processed_text_vector,ipc_article_vector,cosine_ipc
1,3,1,ALTAMAS KABIRJ Leave granted The question whe...,Whoever commits criminal breach of trust shall...,"[0.0007969142, 0.00046393072, 2.7258873e-05, -...","[-0.00023069387, 0.00071619643, 0.00034159506,...",0.021603
2,3,2,ALTAMAS KABIRJ Leave granted The question whe...,Whoever commits murder shall be punished with ...,"[0.0007969142, 0.00046393072, 2.7258873e-05, -...","[-0.0003499089, -0.00018865854, 7.2042945e-05,...",-0.007825
3,3,3,ALTAMAS KABIRJ Leave granted The question whe...,Whoever cheats and thereby dishonestly induces...,"[0.0007969142, 0.00046393072, 2.7258873e-05, -...","[-0.00019394624, -0.0007354331, -5.9539318e-05...",-0.023196
4,3,4,ALTAMAS KABIRJ Leave granted The question whe...,Whoever except in the case provided for by sec...,"[0.0007969142, 0.00046393072, 2.7258873e-05, -...","[-0.0009890613, 0.0004525602, -0.0004944286, -...",0.077747
6,10,1,With Criminal Appeal No227 of 1997 K Sabharwal...,If any person commits suicide whoever abets th...,"[0.00033432688, 0.00021457541, 7.8538775e-05, ...","[0.0001514299, -0.00018431056, 0.00034382497, ...",0.016069
...,...,...,...,...,...,...,...
1584,5963,4,criminal appellate jurisdiction criminal appea...,Whoever intentionally gives false evidence in ...,"[-0.00032380052, -6.8679605e-05, -0.0003186697...","[0.0001682626, 0.0009655272, 0.00075887237, -0...",0.012372
1586,6009,1,criminal appellate jurisdiction criminal appea...,Except in the Chapters and sections mentioned ...,"[0.00048108122, -0.00068164687, 0.00025635207,...","[7.00891e-05, 3.899932e-06, 0.00031869698, 0.0...",0.036863
1588,6010,1,criminal appellate jurisdiction criminal appea...,Whoever commits murder shall be punished with ...,"[-0.00048697312, 0.0004895051, 0.00068926363, ...","[-0.0003499089, -0.00018865854, 7.2042945e-05,...",-0.042196
1589,6010,2,criminal appellate jurisdiction criminal appea...,Whoever knowing or having reason to believe th...,"[-0.00048697312, 0.0004895051, 0.00068926363, ...","[-9.098756e-05, -0.00020535123, 0.0009841664, ...",0.008375


In [21]:
const

,article_number,level_1,processed_text,constitution_article,processed_text_vector,constitution_vector,cosine_constitution
1,0,1,F NARIMAN J Leave granted In 2008 the Punjab ...,Equality of opportunity in matters of public e...,"[-0.0009831183, -0.00059345027, -0.0004421392,...","[0.000796804, 0.0008490001, -0.0002705087, 0.0...",-0.000198
2,0,2,F NARIMAN J Leave granted In 2008 the Punjab ...,Equality before law The State shall not deny t...,"[-0.0009831183, -0.00059345027, -0.0004421392,...","[-0.0005026783, 5.749643e-05, 0.000522226, -0....",-0.022057
4,1,1,S THAKUR J Leave granted These appeals are di...,Recruitment and conditions of service of perso...,"[-0.00039040387, -0.0008521161, -0.00084340497...","[-0.0005202818, -0.0009163675, 0.00022355306, ...",0.104040
5,1,2,S THAKUR J Leave granted These appeals are di...,Enforcement of decrees and orders of Supreme C...,"[-0.00039040387, -0.0008521161, -0.00084340497...","[-0.00047101796, -0.00079761556, -2.8206468e-0...",0.018013
6,1,3,S THAKUR J Leave granted These appeals are di...,Power of Parliament to amend the Constitution ...,"[-0.00039040387, -0.0008521161, -0.00084340497...","[-0.0007636164, 0.0009087316, 0.00018409168, 0...",-0.007085
...,...,...,...,...,...,...,...
5735,5075,1,1999 3 Suppl SCR 597 The Judgment of the C...,Rights of citizenship of certain persons who h...,"[0.0005851183, 0.0006615196, -0.00056849385, -...","[-0.00089419045, -0.000107291875, 0.0003528841...",-0.083147
5736,5075,2,1999 3 Suppl SCR 597 The Judgment of the C...,Definition In this Constitution unless the con...,"[0.0005851183, 0.0006615196, -0.00056849385, -...","[3.7139802e-05, -0.00085376855, 0.00015696377,...",-0.029315
5738,5077,1,1999 1 SCR 104 The Judgments of the Court w...,Protection against arrest and detention in cer...,"[4.2291405e-05, 0.0008433101, -0.00038924543, ...","[0.00072815997, 0.00039746365, -0.00035662059,...",0.010880
5739,5077,2,1999 1 SCR 104 The Judgments of the Court w...,Power of High Courts to issue certain writs No...,"[4.2291405e-05, 0.0008433101, -0.00038924543, ...","[-0.0008169097, 0.0009687003, -0.00067808304, ...",0.049734


In [24]:
pre_final

,article_number,processed_text,label,split,processed_text_vector
0,0,F NARIMAN J Leave granted In 2008 the Punjab ...,1,train,"[-0.0009832437, -0.0006033854, -0.00044061168,..."
1,1,S THAKUR J Leave granted These appeals are di...,0,train,"[-0.00038255434, -0.00082796277, -0.0008486368..."
2,2,Markandey Katju J Leave granted Heard learned...,1,train,"[-0.00016038705, -0.00024669245, 0.0006813637,..."
3,3,ALTAMAS KABIRJ Leave granted The question whe...,1,train,"[0.0007969142, 0.00046393072, 2.7258873e-05, -..."
4,4,CIVIL APPEAL NO 598 OF 2007 K MATHUR J This a...,1,train,"[0.00019346895, -0.00025453963, -0.00034809072..."
...,...,...,...,...,...
6071,6071,civil appellate jurisdiction civil appeal numb...,1,dev,"[0.0008156542, 0.0009509789, 7.1883915e-05, -0..."
6072,6072,criminal appellate jurisdiction special leave\...,0,dev,"[0.0008692498, 0.0006644974, -3.621268e-05, -0..."
6073,6073,civil appellate jurisdiction civil appeal numb...,0,dev,"[-0.00030640335, -0.00033959406, -0.0001059346..."
6074,6074,civil appellate jurisdiction civil appeal numb...,1,dev,"[0.00024632944, 0.0006550132, 1.3497829e-05, 0..."


In [22]:
final_dataframe = pd.DataFrame(
    columns=[
        'article_number',
        'processed_text', 'processed_text_vector',
        'constitution_article', 'constitution_article_vector', 'cosine_constitution',
        'crpc_article', 'crpc_article_vector', 'cosine_crpc',
        'ipc_article', 'ipc_article_vector', 'cosine_ipc'
    ]
)
for i, row in pre_final.iterrows():
    processed_text = row['processed_text']
    processed_text_vector = row['processed_text_vector']

    constitution_article, constitution_article_vector, cosine_constitution = [], [], []
    for j, row_j in const[const['article_number'] == i].iterrows():
        constitution_article.append(row_j['constitution_article'])
        constitution_article_vector.append(row_j['constitution_vector'])
        cosine_constitution.append(row_j['cosine_constitution'])

    for number in range(len(constitution_article)):
        final_dataframe.concat(pd.DataFrame({
            'article_number':number,
            'processed_text': processed_text,
            'processed_text_vector': processed_text_vector,
            'constitution_article': constitution_article[number],
            'constitution_article_vector': constitution_article_vector[number],
            'cosine_constitution': cosine_constitution[number],
        }, index=[0]))

In [17]:
final = pre_final.merge(const[['index', 'level_1', 'constitution_article', 'constitution_vector', 'cosine_constitution']],  how='left', on='index').merge(crpc[['index', 'level_1', 'crpc_article', 'crpc_article_vector', 'cosine_crpc']], how='left', on='index').merge(ipc[['index', 'level_1', 'ipc_article', 'ipc_article_vector', 'cosine_ipc']], how='left', on='index')

In [22]:
import ast
from pandarallel import pandarallel
def vector_to_col(row):
    counter = 0
    for r in row['processed_text_vector']:
        row[f'feat_{counter}'] = r
        counter = counter + 1
    return row

pandarallel.initialize()
final[['index', 'label', 'processed_text_vector', 'cosine_constitution', 'cosine_crpc', 'cosine_ipc']].parallel_apply(vector_to_col, axis=1).drop(['processed_text_vector'], axis=1).to_csv('vector_feat_final_dataset.csv', index=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [19]:
final.drop(['processed_text_vector'], axis=1).to_csv('vector_feat_final_dataset.csv', index=False)

In [106]:
crpc

,index,level_1,processed_text,crpc_article,processed_text_vector,crpc_article_vector,cosine_crpc
1,2,1,Markandey Katju J Leave granted Heard learned...,Order for maintenance of wives children and pa...,"[-0.0005317618, -0.0009307947, -0.000773069, 0...","[0.00045526022, 0.0004528842, 0.00056676846, -...",-0.103867
3,3,1,ALTAMAS KABIRJ Leave granted The question whe...,Compounding of offences The offences punishabl...,"[7.093692e-05, -0.0003043275, 0.00033422757, 9...","[-0.0007361522, -0.00090829184, 0.00061640725,...",0.012644
4,3,2,ALTAMAS KABIRJ Leave granted The question whe...,Saving of inherent powers of High Court Nothin...,"[7.093692e-05, -0.0003043275, 0.00033422757, 9...","[-5.1101564e-05, -8.009684e-05, -0.0006225439,...",-0.091344
6,13,1,2002 Supp 2 SCR 31 The Judgment of the Court...,Recording of confessions and statements Any Me...,"[-0.00011069417, 0.00018372388, 0.00036871334,...","[-0.00095464016, 0.00015591929, 0.00051073177,...",-0.029100
8,24,1,ORIGINAL JURISDICTION Petition No 54 of 1953 P...,Summary procedure for trial for giving false e...,"[-0.0009791156, 0.0008605786, 0.0002155518, 0....","[-0.000157173, -0.0009438658, 0.00032510154, -...",0.012434
...,...,...,...,...,...,...,...
1148,5928,2,criminal appellate jurisdiction criminal appea...,Offence committed outside India When an offenc...,"[-0.0002092112, -0.00017989137, 0.000980109, 0...","[-0.0006591987, 0.0002869164, 0.0006837218, 0....",0.000664
1150,5956,1,criminal appellate jurisdiction criminal appea...,Power to take bond for appearance When any per...,"[0.00097240525, -0.00043522933, -0.00078132737...","[-0.0007090347, -0.0009104842, -2.0453095e-05,...",0.038720
1152,5995,1,criminal appellate jurisdiction criminal appea...,Practising pleader not to sit as Magistrate in...,"[0.00041688836, -0.0008711914, -0.0006943287, ...","[-0.0005956003, 0.0006103989, -0.0005998513, 0...",0.036861
1154,5996,1,criminal appellate jurisdiction criminal appea...,,"[-0.0009752538, -0.0005612226, -0.00012311572,...","[0.00023030984, 0.00077097927, -7.537317e-05, ...",0.144835


In [ ]:
pd.read_csv('')